In [ ]:
# Chargement des librairies

import xml.etree.ElementTree as ET

In [ ]:
# Le fichier d'entrée et de sortie

xml_filename = 'file.xml'
xml_finename_saved = "corrected_file.xml"

In [ ]:
# Parsing du document

tree = ET.parse(xml_filename)
root = tree.getroot()
pages = root[0][0]

In [ ]:
# Boucle sur toutes les pages du document

for idpage,one_page in enumerate(pages):
    
    # Extraction des noeuds de la page
    nodes_one_page = list(one_page)
    tags_one_page = [item.tag for item in nodes_one_page]
    
    # Modification à apporter?
    cdtb = "note" in tags_one_page
    cdta = "texte" in tags_one_page
    cdtfin = cdta and cdtb
    
    # Si modification à apporter, alors ...
    if cdtfin:

        
        
        
        
        # * * * EXTRACTION DES INFORMATIONS PERTINENTES DE LA PAGE * * *
        
        
        
        # Informations des nodes
        note_node = [item for item in nodes_one_page if item.tag == "note"][0]
        texte_node = [item for item in nodes_one_page if item.tag == "texte"][0]
        raw_text = list(note_node.itertext())
        keys_text = [a.text for a in list(note_node)]
        
        # Extraction et structuration des informations du noeud 'note'
        X = []
        for idk,key in enumerate(keys_text):
            for idx,item in enumerate(raw_text):
                if key == item:
                    X.append((key,idx))
        X.append((None,len(raw_text)))
        d = {}
        for idx,item in enumerate(X):
            if item[0] is not None:
                start = item[-1]
                end = X[idx+1][-1]
                d[item[0]] = " ".join(raw_text[start+1:end])
                
                
                
                
                
        # * * * MODIFICATION DU FICHIER XML * * * 
        
        
        
        # Modification du noeud 'texte'                
        for i in list(texte_node.iter()):
            if i.tag == "sup":
                if i.text in d:
                    i.tag = "note"
                    i.set("n",i.text)
                    subnode = ET.fromstring("<sup>" + i.text + "</sup>")
                    subnode.tail = d[i.text]
                    i.append(subnode)
                    i.text = ""

                    
        # Suppression du noeud 'note'
        for nodetoremove in one_page.findall('note'):
            one_page.remove(nodetoremove)

In [ ]:
# Ecriture du document xml modifié

tree.write(xml_finename_saved)